In [2]:
#!pip install selenium

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3

import requests
import json
from selenium import webdriver

In [91]:
url = "https://www.transfermarkt.com/heung-min-son/profil/spieler/91845" #손흥민
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 모든 div 태그 출력
divs = soup.find_all('div')
print(f"Total divs found: {len(divs)}")

# 각 div의 클래스 이름 확인
for idx, div in enumerate(divs):
    print(f"Div {idx}: {div.get('class')}")

Total divs found: 141
Div 0: None
Div 1: ['werbung-skyscraper-left-bound-container']
Div 2: ['werbung-skyscraper-container']
Div 3: ['werbung', 'ad-placement-note', 'werbung-skyscraper']
Div 4: None
Div 5: ['werbung-skyscraperbtf-container']
Div 6: None
Div 7: ['tm-header__box']
Div 8: ['domin-swicher']
Div 9: ['tm-header__search']
Div 10: ['tm-header__login']
Div 11: ['main-navbar__details']
Div 12: ['hamburger']
Div 13: ['dropdown-layer']
Div 14: ['main-navbar__dropdown']
Div 15: ['recommendation']
Div 16: ['tm-discover__hero-container', 'tm-discover__hero-container-grid', 'tm-discover-container-grid']
Div 17: ['tm-link-list-element']
Div 18: ['quick-select-wrapper']
Div 19: ['werbung', 'ad-placement-note', 'werbung-billboard']
Div 20: None
Div 21: ['modal', 'micromodal-slide']
Div 22: ['modal__overlay']
Div 23: ['modal__container']
Div 24: ['modal__content']
Div 25: ['data-header__headline-container']
Div 26: ['data-header__badge-container']
Div 27: ['data-header__box--big']
Div 28:

In [40]:
url = "https://www.transfermarkt.com/ashley-young/profil/spieler/14086" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# info-table 내의 모든 span 요소 추출
table_data = soup.find('div', class_='info-table')
spans = table_data.find_all('span', class_='info-table__content')

# 내부 텍스트와 하위 태그 처리 함수
def get_text_or_child_content(tag):
    # 텍스트가 있는 경우
    if tag.text.strip():
        return tag.text.strip()
    # 텍스트가 없으면 내부 태그 확인
    elif tag.find('a'):  # 링크가 있는 경우
        return tag.find('a').text.strip()
    elif tag.find('img'):  # 이미지가 있는 경우
        return tag.find('img').get('alt', '').strip()
    elif tag.find('span'):  # 중첩된 span
        return tag.find('span').text.strip()
    return "N/A"  # 데이터가 없으면 "N/A" 반환

# 데이터를 쌍으로 저장
data = []
current_key = None
for span in spans:
    if 'info-table__content--regular' in span['class']:  # key
        current_key = get_text_or_child_content(span)
    elif 'info-table__content--bold' in span['class'] and current_key:  # value
        value = get_text_or_child_content(span)
        data.append((current_key, value))
        current_key = None  # 매칭된 후 초기화

# Pandas DataFrame 변환
df = pd.DataFrame(data, columns=['Attribute', 'Value'])

# DataFrame 출력
print(df)


                   Attribute                 Value
0      Name in home country:    Ashley Simon Young
1         Date of birth/Age:      Jul 9, 1985 (39)
2            Place of birth:             Stevenage
3                    Height:                1,75 m
4               Citizenship:      England  Jamaica
5                  Position:  Defender - Left-Back
6                      Foot:                  both
7              Player agent:          CAA Base Ltd
8              Current club:            Everton FC
9                    Joined:          Jul 13, 2023
10         Contract expires:          Jun 30, 2025
11  Last contract extension:          Jun 25, 2024
12                Outfitter:                adidas
13             Social-Media:                      


tm-transfer-history( 이적 history)는 동적으로 로드되는 데이터 ㅜㅜ
<section class = "quick-fact"> (market value 순위)도 동적 로드 데이터



In [95]:
#main position, other position 정보 가져오기

# 특정 클래스를 가진 데이터 가져오기
table2 = soup.find('div', {
    'class': 'detail-position__box'
})
#print(table2)

# Main position 추출
main_position = soup.find('dt', text='Main position:').find_next('dd').text

# Other positions 추출
other_positions = soup.find('dt', text='Other position:').find_next_siblings('dd')
other_positions = [pos.text for pos in other_positions]

# Pandas DataFrame 생성
data = {
    'Position Type': ['Main position', 'Other position'],
    'Positions': [main_position, ', '.join(other_positions)]
}

df = pd.DataFrame(data)

# DataFrame 출력
print(df)

<div class="detail-position__box">
<div class="detail-position__inner-box">
<dl>
<dt class="detail-position__title">Main position:</dt>
<dd class="detail-position__position">Left Winger</dd>
</dl>
</div>
<div class="detail-position__position">
<dl>
<dt class="detail-position__title">Other position:</dt>
<dd class="detail-position__position">Centre-Forward</dd>
<dd class="detail-position__position">Second Striker</dd>
</dl>
</div>
</div>
    Position Type                       Positions
0   Main position                     Left Winger
1  Other position  Centre-Forward, Second Striker


/var/folders/fz/7mv6m3856xn521zxcqxvp2gr0000gn/T/ipykernel_33281/3202429113.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  main_position = soup.find('dt', text='Main position:').find_next('dd').text
/var/folders/fz/7mv6m3856xn521zxcqxvp2gr0000gn/T/ipykernel_33281/3202429113.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  other_positions = soup.find('dt', text='Other position:').find_next_siblings('dd')


In [37]:

url = "https://www.transfermarkt.com/jacob-ramsey/marktwertverlauf/spieler/503749"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# <section class="quick-fact"> 요소 가져오기
#facts = soup.find('section', {'class': 'quick-fact'})

# 섹션 내용 출력 (전체 HTML 포함)
#if section:
#    print(section.prettify())
#else:
#    print("Section with class 'quick-fact' not found.")

# BeautifulSoup 객체 생성
#soup = BeautifulSoup(html_content, 'html.parser')

# 모든 a 태그를 가져오기
facts = soup.find_all('a', class_='quick-fact__link')

# 데이터 저장용 리스트
data = []

# 필요한 정보 추출
for fact in facts:
    h3_text = fact.find('h3', class_='quick-fact__headline').get_text(strip=True)
    rank = fact.find('span', class_='quick-fact__content--large').get_text(strip=True)
    footnote = fact.find('p', class_='quick-fact__footnote').get_text(strip=True)
    data.append({'Category': h3_text, 'Rank': rank, 'Description': footnote})

# Pandas DataFrame 생성
df = pd.DataFrame(data)

# 결과 출력
print(df)

         Category Rank                                        Description
0       Worldwide  239      Ranked number 239 among all players worldwide
1  Premier League  112  Ranked number 112 among all players of the Pre...
2     Aston Villa    9   Ranked number 9 among all players at Aston Villa
3         England   33    Ranked number 33 among all players from England
4   Left Midfield    1  Ranked number 1 among all players playing as "...
5       Year 2001   31       Ranked number 31 among all playersborn 2001.


In [39]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.transfermarkt.com/tyrone-mings/nationalmannschaft/spieler/253677"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# <section class="quick-fact"> 요소 가져오기
#facts = soup.find('section', {'class': 'quick-fact'})

# 섹션 내용 출력 (전체 HTML 포함)
#if section:
#    print(section.prettify())
#else:
#    print("Section with class 'quick-fact' not found.")

# BeautifulSoup 객체 생성
#soup = BeautifulSoup(html_content, 'html.parser')

# 모든 a 태그를 가져오기
facts = soup.find_all('div', class_='quick-fact__link')


# <div class="box"> 내의 <table> 찾기
box_div = soup.find('div', class_='box')
table = box_div.find('table')
#print(table)


# 테이블 헤더 추출
headers = [header.get_text(strip=True) for header in table.find_all('th')]

# 테이블 본문 추출
rows = []
for row in table.find('tbody').find_all('tr'):
    # 각 행의 셀 데이터 추출
    cells = [cell.get_text(strip=True) for cell in row.find_all(['td', 'th'])]
    if cells:  # 비어있는 행 제외
        rows.append(cells)

# 데이터프레임 생성 시 열 수 조정
# 데이터가 헤더보다 열이 많다면, 초과 열을 잘라냄
max_columns = len(headers)
adjusted_rows = [row[:max_columns] for row in rows]

# Pandas DataFrame 생성
df = pd.DataFrame(adjusted_rows, columns=headers[:max_columns])

# 데이터프레임 출력
print(df)

    # National team    Debut                     Coach at debut  \
0           England     None          None  None           None   
1  15                England  Oct 14, 2019    18              2   

       Age at debut  
0              None  
1  Gareth Southgate  


In [115]:
import requests
from bs4 import BeautifulSoup

# URL 및 요청 설정
url = "https://www.transfermarkt.com/heung-min-son/meistetorbeteiligungen/spieler/91845"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# <div id="statistics-table-defensive"> 추출
target_div = soup.find('div', {'id': 'player-matches-table'})
print(target_div)

# # 하위 HTML 요소 추출
# if target_div:
#     all_children = target_div.find_all()  # 모든 하위 요소를 리스트로 반환
#     html_elements = [str(child) for child in all_children]  # 문자열로 변환
#     for element in html_elements:
#         print(element)
# else:
#     print("Target div not found.")


None
